<a href="https://colab.research.google.com/github/fabriciobarili/LLM_RAG/blob/main/LLM_Fine_tuning_RAG_LANGCHAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-openai
!pip install -U langchain langchain-community openai faiss-cpu
!pip install tiktoken
!pip install sentence-transformers
!pip install chromadb

In [ ]:
from google.colab import userdata

userdata.get('OPENAI_API_KEY')


In [ ]:
import os

api_key = userdata.get('OPENAI_API_KEY')

print(api_key is not None)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
import os

api_key = userdata.get('OPENAI_API_KEY')

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set")

llm = ChatOpenAI(model_name="gpt-4", temperature=0.6, openai_api_key=api_key)

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.csv_loader import CSVLoader
import os

# Specify the directory containing the CSV files
directory_path = '/content/drive/MyDrive/LLM_DOCS/'  # Replace with your directory

# Initialize DirectoryLoader with CSV files filter
loader = DirectoryLoader(directory_path, glob="*.csv", loader_cls=CSVLoader)

# Load the documents
data = loader.load()

# documents will be a list of Document objects, each representing a row from a CSV file.
# You can access the content and metadata of each document.
for doc in data:
    print(doc.page_content)
    print(doc.metadata)

In [ ]:
import tiktoken

tiktoken.encoding_for_model('gpt-4o')

In [ ]:
tokenizer = tiktoken.get_encoding('o200k_base')

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

#tiktoken_len("Os sintomas da depressão podem ser diversos. A melhor maneira é consultar um profissional capacitado")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

In [ ]:
chunks = text_splitter.split_documents(data)

len(chunks)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
embed = hf.embed_documents(texts=['h','e'])

print(len(embed[1]))

In [ ]:
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(documents=chunks, embedding=hf)

In [ ]:
vectordb.similarity_search('Principais casos de hipertensão', k=3)

In [ ]:
#CID
"""from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
import os

api_key = userdata.get('OPENAI_API_KEY')


print(api_key is not None)

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set")

llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.1, openai_api_key=api_key)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

p = []
p.append("Um paciente relata: falta de ar progressiva há meses, piora ao realizar esforço físico. Queixa-se de tosse crônica com secreção esbranquiçada, principalmente pela manhã . E o médico observa  Ausculta pulmonar revela roncos e sibilos bilaterais. Saturação de oxigênio em repouso de 89%. Histórico de tabagismo intenso (40 anos-maço).. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: Queixa-se de cansaço constante ao caminhar pequenas distâncias. Episódios frequentes de tosse produtiva, principalmente ao acordar . E o médico observa  RX de tórax demonstra hiperinsuflação pulmonar. Espirometria confirma obstrução ao fluxo aéreo. Uso contínuo de broncodilatadores relatado.. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: sede excessiva, perda de peso sem motivo aparente e aumento da frequência urinária nas últimas semanas . E o médico observa  Glicemia em jejum de 178 mg/dL. Hemoglobina glicada (HbA1c) de 8,2%. IMC de 31 kg/m². Familiar de primeiro grau com histórico de diabetes.. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: visão embaçada e formigamento nos pés, além de ferida em membro inferior que demora a cicatrizar . E o médico observa  Exame de sensibilidade periférica alterado. Presença de ulceração em hálux direito. Glicemia pós-prandial 250 mg/dL.. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: tristeza persistente, desânimo, perda de interesse nas atividades diárias e dificuldades para dormir há mais de um mês . E o médico observa  Afeto deprimido, fala lenta e pensamento pessimista. Escala de depressão de PHQ-9 com pontuação de 32 (grave).. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: Queixa de cansaço constante, choro frequente e sentimento de inutilidade. Dificuldade para se concentrar no trabalho . E o médico observa  Relato de isolamento social. Apetite diminuído. Sem sinais de psicose. Diagnóstico anterior de episódio depressivo confirmado.. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: dores de cabeça frequentes pela manhã, principalmente na nuca. Diz sentir tontura ao levantar rapidamente . E o médico observa  Pressão arterial aferida em 162/98 mmHg. Exames laboratoriais indicam função renal preservada. ECG com sinais discretos de hipertrofia ventricular esquerda.. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: histórico familiar de hipertensão e relata que tem sentido o “coração bater forte” em momentos de estresse . E o médico observa  Aferição pressórica mostra média de 150/95 mmHg em três ocasiões distintas. Índice de massa corporal elevado (IMC = 29,8).. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: comparecer para avaliação de rotina. Relata estar se sentindo bem, sem queixas no momento . E o médico observa  Sinais vitais dentro da normalidade. Exames laboratoriais normais. Índice de massa corporal dentro da faixa ideal.. Para esta condição, qual CID você sugere?")
p.append("Um paciente relata: vir apenas para consulta preventiva. Nega sintomas atuais ou doenças prévias. Alimentação e sono considerados adequados . E o médico observa  Exame físico sem alterações. Pressão arterial de 118/76 mmHg, frequência cardíaca de 72 bpm. Glicemia de jejum 88 mg/dL.. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Mulher de 57 anos, relata:  nunca ter fumado, tosse constante e incapacidade de realizar atividades fisicas e o médico observa Exame mostra SpO2 a 90%, sibilos leves, e aumento da frequência respiratória.. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Homem de 43 anos, relata: ser fumante ativo, apresenta tosse constante com catarro e falta de ar e o médico observa SpO2 a 83% com crepitações bilaterais e uso de musculos acessórios par auxílio na respiração. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Mulher 21 anos,  relata: estar em boas condições de saúde, não pratica esportes e o médico observa Exames iniciais demonstraram sinais vitais, reflexo e pressão normal com colesterol um pouco acima da média. Para esta condição, qual CID você sugere?")
p.append("Um paciente, homem de 50 anos, relata: ter atividades fisicas regularmente, sono de 8h por dia e o médico observa Exames apresentam bom tônus muscular e e reflexos normais, exames de sangue apresentaram resultados dentro do esperado. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Adolescente de 17 anos, relata: queixa de dores de cabeça e tontura. possuí historico de obesidade na família e o médico observa Apresenta pressão arterial de 155/95 mmHg e exame neurológico normal. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Homem de 55 anos, relata: queixas de dores no peito e totura ao realizar atividade física e o médico observa Frequência cardíaca de 59BPM e pressão arterial de 170/140 mmHg. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Mulher de 30 anos, grávida, relata: aumento de sede e fadiga e o médico observa Glicemia em jejum: 160mg/dL. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Homem de 63 anos, relata: relata formigamento nos pés e miccção frequente e o médico observa Glicemia em jejum de 175mg/dL com histórico familiar de diabetes e cultura urinária positiva para E.Coli. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Jovem de 19 anos, relata: falta de energia e relatando grande desanimo e sentimento de 'vazio' e o médico observa Paciente com fala lenta e baixa, com quadro de perda de peso. Para esta condição, qual CID você sugere?")
p.append("Um paciente, Idoso de 78 anos viúvo, relata: sentimento de tristeza após perda de familiar próximo e falta de apetite e o médico observa fala lenta, choro durante a consulta e desanimo para realizar atividades diárias. Para esta condição, qual CID você sugere?")


for i in p:
  response = qa_chain({'query':i})
  print("-----------------------------")
  print(f"Pergunta: {response['query']}")
  print(f"Resposta: {response['result']}")
  print("Fim da Resposta")
  print("-----------------------------")
  print('\n\n')"""

In [ ]:
#Perguntas com o Plano de ação

from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
import os

api_key = userdata.get('OPENAI_API_KEY')


print(api_key is not None)

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set")

llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.1, openai_api_key=api_key)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

p = []
p.append("Um paciente relata: falta de ar progressiva há meses, piora ao realizar esforço físico. Queixa-se de tosse crônica com secreção esbranquiçada, principalmente pela manhã . E o médico observa  Ausculta pulmonar revela roncos e sibilos bilaterais. Saturação de oxigênio em repouso de 89%. Histórico de tabagismo intenso (40 anos-maço).. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: Queixa-se de cansaço constante ao caminhar pequenas distâncias. Episódios frequentes de tosse produtiva, principalmente ao acordar . E o médico observa  RX de tórax demonstra hiperinsuflação pulmonar. Espirometria confirma obstrução ao fluxo aéreo. Uso contínuo de broncodilatadores relatado.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: sede excessiva, perda de peso sem motivo aparente e aumento da frequência urinária nas últimas semanas . E o médico observa  Glicemia em jejum de 178 mg/dL. Hemoglobina glicada (HbA1c) de 8,2%. IMC de 31 kg/m². Familiar de primeiro grau com histórico de diabetes.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: visão embaçada e formigamento nos pés, além de ferida em membro inferior que demora a cicatrizar . E o médico observa  Exame de sensibilidade periférica alterado. Presença de ulceração em hálux direito. Glicemia pós-prandial 250 mg/dL.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: tristeza persistente, desânimo, perda de interesse nas atividades diárias e dificuldades para dormir há mais de um mês . E o médico observa  Afeto deprimido, fala lenta e pensamento pessimista. Escala de depressão de PHQ-9 com pontuação de 32 (grave).. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: Queixa de cansaço constante, choro frequente e sentimento de inutilidade. Dificuldade para se concentrar no trabalho . E o médico observa  Relato de isolamento social. Apetite diminuído. Sem sinais de psicose. Diagnóstico anterior de episódio depressivo confirmado.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: dores de cabeça frequentes pela manhã, principalmente na nuca. Diz sentir tontura ao levantar rapidamente . E o médico observa  Pressão arterial aferida em 162/98 mmHg. Exames laboratoriais indicam função renal preservada. ECG com sinais discretos de hipertrofia ventricular esquerda.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: histórico familiar de hipertensão e relata que tem sentido o “coração bater forte” em momentos de estresse . E o médico observa  Aferição pressórica mostra média de 150/95 mmHg em três ocasiões distintas. Índice de massa corporal elevado (IMC = 29,8).. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: comparecer para avaliação de rotina. Relata estar se sentindo bem, sem queixas no momento . E o médico observa  Sinais vitais dentro da normalidade. Exames laboratoriais normais. Índice de massa corporal dentro da faixa ideal.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente relata: vir apenas para consulta preventiva. Nega sintomas atuais ou doenças prévias. Alimentação e sono considerados adequados . E o médico observa  Exame físico sem alterações. Pressão arterial de 118/76 mmHg, frequência cardíaca de 72 bpm. Glicemia de jejum 88 mg/dL.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Mulher de 57 anos, relata:  nunca ter fumado, tosse constante e incapacidade de realizar atividades fisicas e o médico observa Exame mostra SpO2 a 90%, sibilos leves, e aumento da frequência respiratória.. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Homem de 43 anos, relata: ser fumante ativo, apresenta tosse constante com catarro e falta de ar e o médico observa SpO2 a 83% com crepitações bilaterais e uso de musculos acessórios par auxílio na respiração. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Mulher 21 anos,  relata: estar em boas condições de saúde, não pratica esportes e o médico observa Exames iniciais demonstraram sinais vitais, reflexo e pressão normal com colesterol um pouco acima da média. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, homem de 50 anos, relata: ter atividades fisicas regularmente, sono de 8h por dia e o médico observa Exames apresentam bom tônus muscular e e reflexos normais, exames de sangue apresentaram resultados dentro do esperado. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Adolescente de 17 anos, relata: queixa de dores de cabeça e tontura. possuí historico de obesidade na família e o médico observa Apresenta pressão arterial de 155/95 mmHg e exame neurológico normal. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Homem de 55 anos, relata: queixas de dores no peito e totura ao realizar atividade física e o médico observa Frequência cardíaca de 59BPM e pressão arterial de 170/140 mmHg. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Mulher de 30 anos, grávida, relata: aumento de sede e fadiga e o médico observa Glicemia em jejum: 160mg/dL. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Homem de 63 anos, relata: relata formigamento nos pés e miccção frequente e o médico observa Glicemia em jejum de 175mg/dL com histórico familiar de diabetes e cultura urinária positiva para E.Coli. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Jovem de 19 anos, relata: falta de energia e relatando grande desanimo e sentimento de 'vazio' e o médico observa Paciente com fala lenta e baixa, com quadro de perda de peso. Para esta condição, qual plano de ação você sugere?")
p.append("Um paciente, Idoso de 78 anos viúvo, relata: sentimento de tristeza após perda de familiar próximo e falta de apetite e o médico observa fala lenta, choro durante a consulta e desanimo para realizar atividades diárias. Para esta condição, qual plano de ação você sugere?")


for i in p:
  response = qa_chain({'query':i})
  print("-----------------------------")
  print(f"Pergunta: {response['query']}")
  print(f"Resposta: {response['result']}")
  print("Fim da Resposta")
  print("-----------------------------")
  print('\n\n')

In [ ]:
for i in p:
  print(i)